Prepare csv files downloaded from PubChem database

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Manuscrip_preparation/compounds/GLP1R_data2.csv')
df

,Unnamed: 0,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME
0,0,44290899.0,CC[C@H](C)[C@@H](C(=O)N[C@@H](C)C(=O)N[C@@H](C...,Active
1,1,122189729.0,CC[C@H](C)[C@@H](C(=O)N[C@@H](C)C(=O)N[C@@H](C...,Active
2,2,122189730.0,CCCCCCCCCCCCCCCCCC(=O)N[C@H](CCC(=O)NCCCC[C@@H...,Active
3,3,122189731.0,CCCCCCCCCCCCCCCCCCCC(=O)N[C@@H](CCC(=O)NCCOCCO...,Active
4,4,122189732.0,CC[C@H](C)[C@@H](C(=O)N[C@@H](C)C(=O)N[C@@H](C...,Active
...,...,...,...,...
862,862,604525.0,CC1=C(C(=NN1CC(=O)NN)C)Br,Inactive
863,863,2961794.0,CC(C(=O)NC(CC1=CC=C(C=C1)O)C(=O)OC)OC2=CC=CC=C2,Inactive
864,864,752521.0,CC1(N=C(NC(=N1)NC2=CC(=CC=C2)Br)N)C,Inactive
865,865,12006165.0,CCSC1=NN=C(C(=O)N1COC(=O)C2=CC=C(C=C2)S(=O)(=O...,Inactive


Descriptors

In [ ]:
#downloading libraries
!pip install rdkit-pypi
!pip install mordred

from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

from mordred import Calculator, descriptors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 34.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.3 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176723 sha256=7cfb2ac1973cf49bcc1fbb971b436376471a494bd097ec3ac2be6c627a4d6ddc
  Stored in directory: /root/.cache/pip/wheels/05/95/d1/9e913738f0e8362b3676917b953a60afd76d2b0b99ff8a71ec
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [ ]:
df2 = pd.concat( [df['PUBCHEM_EXT_DATASOURCE_SMILES'], df['PUBCHEM_CID']], axis=1 )  #extract smiles and cid columns from dataframe and add to df2
df2.to_csv('/Manuscrip_preparation/compounds/molecule.smi', sep='\t', index=False, header=False) #save to csv file without headers and index
df2.to_csv('/Manuscrip_preparation/compounds/molecule2.smi', sep='\t', index=False, header=False)
df2

,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_CID
0,CC[C@H](C)[C@@H](C(=O)N[C@@H](C)C(=O)N[C@@H](C...,44290899.0
1,CC[C@H](C)[C@@H](C(=O)N[C@@H](C)C(=O)N[C@@H](C...,122189729.0
2,CCCCCCCCCCCCCCCCCC(=O)N[C@H](CCC(=O)NCCCC[C@@H...,122189730.0
3,CCCCCCCCCCCCCCCCCCCC(=O)N[C@@H](CCC(=O)NCCOCCO...,122189731.0
4,CC[C@H](C)[C@@H](C(=O)N[C@@H](C)C(=O)N[C@@H](C...,122189732.0
...,...,...
862,CC1=C(C(=NN1CC(=O)NN)C)Br,604525.0
863,CC(C(=O)NC(CC1=CC=C(C=C1)O)C(=O)OC)OC2=CC=CC=C2,2961794.0
864,CC1(N=C(NC(=N1)NC2=CC(=CC=C2)Br)N)C,752521.0
865,CCSC1=NN=C(C(=O)N1COC(=O)C2=CC=C(C=C2)S(=O)(=O...,12006165.0


In [ ]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    Mol_descriptors =[]
    for mol in mols:
        # add hydrogens to molecules
        mol=Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names

# Function call
Mol_descriptors,desc_names = RDkit_descriptors(df2['PUBCHEM_EXT_DATASOURCE_SMILES'])
df_descriptors = pd.DataFrame(Mol_descriptors,columns=desc_names)
df_descriptors

NameError: ignored

In [ ]:
#morgan fingerprints
def morgan_fpts(data):
    Morgan_fpts = []
    for i in data:
        mol = Chem.MolFromSmiles(i)
        fpts =  AllChem.GetMorganFingerprintAsBitVect(mol,2,2048)
        mfpts = np.array(fpts)
        Morgan_fpts.append(mfpts)
    return np.array(Morgan_fpts)

Morgan_fpts = morgan_fpts(df2['PUBCHEM_EXT_DATASOURCE_SMILES'])
Morgan_fingerprints = pd.DataFrame(Morgan_fpts,columns=['Col_{}'.format(i) for i in range(Morgan_fpts.shape[1])])
Morgan_fingerprints

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_2038,Col_2039,Col_2040,Col_2041,Col_2042,Col_2043,Col_2044,Col_2045,Col_2046,Col_2047
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
863,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
864,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#mordred descriptor calculation
def All_Mordred_descriptors(data):
    calc = Calculator(descriptors, ignore_3D=False)
    mols = [Chem.MolFromSmiles(smi) for smi in data]

    df = calc.pandas(mols)
    return df

mordred_descriptors = All_Mordred_descriptors(df2['PUBCHEM_EXT_DATASOURCE_SMILES'])
mordred_descriptors

In [ ]:
#save computed descrip
#mordred_descriptors.to_csv('/Manuscrip_preparation/mordred_descriptors.csv', index= False)
Morgan_fingerprints.to_csv('/Manuscrip_preparation/Morgan_fingerprints.csv', index= False)
#df_descriptors.to_csv('/Manuscrip_preparation/rdkit_descriptors.csv', index= False)